In [1]:
from nlptoolkits.FirmLevelRiskKits import DictionaryT, PreprocessT
from nlptoolkits.SmallKits import IOHandlerT
import nlptoolkits

We make the basic classes for further use

In [2]:
internation_tax_dictionary = IOHandlerT.file_to_list(
    './input_data/annotatedsent_Principles_of_International_Taxation.txt',
    charset_error_encoding='utf-8'
)[:1000]

tweets_texture = IOHandlerT.file_to_list(
    './input_data/tweets_origin.txt',
    charset_error_encoding='utf-8'
)[:1000]

# make a class to clean the data(dict)
corenlp_preprocess_cls = PreprocessT.NgramDataPreprocessor(remove_stopwords_set=nlptoolkits.resources.SET_STOPWORDS)

# make a class to build the dictionary
corenlp_ngram_dict_cls = DictionaryT.NgramDictionaryBuilder(remove_stopwords_set=nlptoolkits.resources.SET_STOPWORDS,
                                                            remove_token_lessequal_then_length=4)

In [3]:
internation_tax_dictionary[:3]

['the[POS:DT] principal[POS:JJ] aim[POS:NN] of[POS:IN] this[POS:DT] book[POS:NN] be[POS:VBZ] to[POS:TO] [NER:DATE]present[POS:VB] the[POS:DT] topic[POS:NN] in[POS:IN] a[POS:DT] accessible[POS:JJ] manner[POS:NN] to[POS:IN] that[POS:DT] who[POS:WP] be[POS:VBP] fairly[POS:RB] new[POS:JJ] to[POS:IN] international[POS:JJ] taxation[POS:NN] or[POS:CC] ,[POS:,] indeed[POS:RB] ,[POS:,] be[POS:VBP] new[POS:JJ] to[POS:IN] taxation[POS:NN] in[POS:IN] any[POS:DT] guise[POS:NN] .[POS:.]',
 'it[POS:PRP] will[POS:MD] be[POS:VB] useful[POS:JJ] for[POS:IN] university[POS:NN][SEP]student[POS:NNS] ,[POS:,] for[POS:IN] that[POS:DT] prepare[POS:VBG] for[POS:IN] professional[POS:JJ] qualification[POS:NNS] ,[POS:,] and[POS:CC] for[POS:IN] practitioner[POS:NNS] generally[POS:RB] .[POS:.]',
 '[NER:PERSON]Angharad[POS:NNP][SEP]Miller[POS:NNP] and[POS:CC] I[POS:PRP] [NER:ORDINAL]first[POS:RB] embark[POS:VBD] on[POS:IN] this[POS:DT] project[POS:NN] in[POS:IN] [NER:DATE]2006[POS:CD] ,[POS:,] at[POS:IN] she[POS:PRP$

In [4]:
tweets_texture[:3]

['"This year, throw kindness around like confetti! Happy New Year!  There is only one Kennan Fritz and we\'re sad to see him go! Kennan, you\'ve been part of the Independence Bank family for almost two decades and in many of our markets. You will be missed by all of us as we celebrate your retirement and wish you well!  REMINDER: Be aware of calls asking you to do anything you wouldn<U+2019>t normally do with your online account/debit card info. Don\'t disclose any personal information unless you\'re calling the bank and initiating. The bank will never call you and initiate the request for information.  Mackenzie Metcalf is prepared and qualified to continue serving clients as a Financial Advisor. Mackenzie is thrilled to advance her career by educating people on products and services offered by what she considers to be the best bank in Kentucky.   Don<U+2019>t forget! If you can<U+2019>t get out or prefer to stay at home we offer many convenient ways to bank from the comfort of your o

Part to make Ngram dictionary

In [5]:
international_tax_tfidf = corenlp_ngram_dict_cls.n_gramizer_dictionary_builder(internation_tax_dictionary, scorer = 'tfidf', 
                                                                               remove_ngram_postags_combinations=None
                                                                               )

100%|██████████| 8280/8280 [00:01<00:00, 4963.67it/s]


In [6]:
international_tax_tfidf

{('chain', 'person'): 9.71534052268532e-05,
 ('rebate', 'allow'): 9.71534052268532e-05,
 ('maximum', 'possible'): 9.71534052268532e-05,
 ('reason', 'relief'): 9.71534052268532e-05,
 ('length', 'ownership'): 9.71534052268532e-05,
 ('market', 'proximity'): 9.71534052268532e-05,
 ('corporate', 'income'): 0.000485767026134266,
 ('achieve', 'tax'): 9.71534052268532e-05,
 ('point', 'rate'): 9.71534052268532e-05,
 ('and', 'investment'): 9.71534052268532e-05,
 ('arise', 'country'): 9.71534052268532e-05,
 ('number', 'design'): 9.71534052268532e-05,
 ('place', 'always'): 9.71534052268532e-05,
 ('investment', 'investment'): 9.71534052268532e-05,
 ('primarily', 'provide'): 9.71534052268532e-05,
 ('staff', 'sixth'): 9.71534052268532e-05,
 ('suggest', 'amount'): 9.71534052268532e-05,
 ('spend', 'tax'): 9.71534052268532e-05,
 ('specially', 'target'): 9.71534052268532e-05,
 ('least', 'provide'): 9.71534052268532e-05,
 ('remain', 'major'): 9.71534052268532e-05,
 ('chapter', 'result'): 9.71534052268532e

You can remove words by remove phrases which are in another dictionary, but we do not do here now.

We now clean the tweets data quickly by using the nltk quick cleaner

In [7]:
PreprocessT.naive_nltk_annotator(
    tweets_texture
)[0]

"`` This year , throw kindness around like confetti ! Happy New Year ! There is only one Kennan Fritz and we 're sad to see him go ! Kennan , you 've been part of the Independence Bank family for almost two decades and in many of our markets . You will be missed by all of us as we celebrate your retirement and wish you well ! REMINDER : Be aware of calls asking you to do anything you wouldn < U+2019 > t normally do with your online account/debit card info . Do n't disclose any personal information unless you 're calling the bank and initiating . The bank will never call you and initiate the request for information . Mackenzie Metcalf is prepared and qualified to continue serving clients as a Financial Advisor . Mackenzie is thrilled to advance her career by educating people on products and services offered by what she considers to be the best bank in Kentucky . Don < U+2019 > t forget ! If you can < U+2019 > t get out or prefer to stay at home we offer many convenient ways to bank from

In [8]:
corenlp_preprocess_cls.clean_annotated_texture_list(PreprocessT.naive_nltk_annotator(
    tweets_texture
), processes=5)[0]

Cleaning the annotated data...


100%|██████████| 1000/1000 [00:12<00:00, 79.47it/s]


'year throw kindness around like confetti happy new year one kennan fritz sad see go kennan part independence bank family almost two decades many markets will missed us celebrate retirement wish well reminder aware calls asking anything wouldn normally online card info disclose personal information unless calling bank initiating bank will never call initiate request information mackenzie metcalf prepared qualified continue serving clients financial advisor mackenzie thrilled advance career educating people products services offered considers best bank kentucky don forget get prefer stay home offer many convenient ways bank comfort home digitalbanking telephonebanking website stay safe everyone independence bank scholarship program accepting applications whether plan attend traditional college college enrolling trade school looking high school seniors want education hearts continue healing devastation destruction caused severe storms one month ago today gifts generosity helping give hop